## 十一、小参数量微调

<img src="peft.png" style="margin-left: 0px" width="600px">

- 定义微调数据集加载器
- 定义数据处理函数
- 加载预训练模型：AutoModel.from_pretrained(MODEL_NAME_OR_PATH)
- 在预训练模型上增加任务相关输出层 （如果需要）
- 加载预训练 Tokenizer：AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
- **定义注入参数的方法（见下文）**
- 定义各种超参
- 定义 Trainer
- 定义 Evaluation Metric
- 开始训练

### 11.1、Prompt Tuning

- 在输入序列前，额外加入一组伪 Embedding 向量
- 只训练这组伪 Embedding，从而达到参数微调的效果

<img src="soft-prompt.png" style="margin-left: 0px" width="800px">

### 11.2、P-Tuning

- 用一个生成器生成上述伪 Embedding
- 只有生成器的参数是可训练的

<img src="pt.png" style="margin-left: 0px" width="800px">

### 11.3、Prefix-Tuning

- 伪造前面的 Hidden States
- 只训练伪造的这个 Prefix

<img src="pt2.png" style="margin-left: 0px" width="800px">

### 11.4、LoRA

- 在 Transformer 的参数矩阵上加一个低秩矩阵（$A\times B$）
- 只训练 A，B
- 理论上可以把上述方法应用于 Transformer 中的任意参数矩阵，包括 Embedding 矩阵
- 通常应用于 Query, Value 两个参数矩阵

<img src="lora.png" style="margin-left: 0px" width="800px">

### 11.5、AdaLoRA

- 不预先指定可训练矩阵的秩
- 根据参数矩阵的重要性得分，在参数矩阵之间自适应地分配参数预算。


## 十二、**实战 1：** 基于 Prefix-Tuning 微调 ChatGLM-6B

### 12.1、数据准备

法律领域阅读理解
https://github.com/china-ai-law-challenge/CAIL2019/tree/master/%E9%98%85%E8%AF%BB%E7%90%86%E8%A7%A3

**样本举例**

判决书:

经审理查明,原、被告于 1987 年结婚,于 1989 年 4 月 18 日生育一女翟 x8,1994 年 5 月 22 日生育一子翟旭雷,女儿翟 x8 系先天性脑瘫原、被告因感情破裂于 2005 年 11 月 7 日在洪雅县民政局协议离婚,同时约定了两个未成年的子女均由翟 x2 抚养,由王 x0 每年支付抚养费 5000 元等主要内容原、被告离婚后,原告王 x0 于 2006 年外出务工至今,期间支付过部分抚养费,被告翟 x2 在抚养翟 x8 时,因翟 x2 在三宝镇场镇经营蛋糕生意,其女儿翟 x8 主要由翟 x2 母亲翟光秀(1936 年 7 月 13 日出生)进行日常照顾 2014 年农历 2 月原告王 x0 外出务工回来去看望其女儿翟 x8 时,见其穿着不整洁,原告为此曾与被告协商变更翟 x8 的抚养关系未果 2014 年 4 月 18 日,原告王 x0 再次来到被告翟 x2 处看望翟 x8,再次与被告协商变更翟 x8 抚养关系未果,双方发生抓扯,后经洪雅县公安局将军派出所调解,原告将翟 x8 带回其住处照顾至今,现原告诉至法院请求将翟 x8 变更为由其抚养另查明,原告王 x0 现居住在洪雅县洪川镇的廉租房,在洪雅县城经营服装生意上述事实,有原、被告陈述,原、被告离婚协议书,证人证言,原告拍摄的翟 x8 的生活照片以及原告提供的对翟 x8 生活状况的视频资料等证据证实

问题：

原告现居住在哪里？

答案：

洪雅县洪川镇的廉租房

### 12.2、ChatGLM 的数据加载方式分析

输入本质上就是一段按格式拼接的文本


```python
def preprocess_function_eval(examples):
    inputs, targets = [], []
    for i in range(len(examples[prompt_column])):
        if examples[prompt_column][i] and examples[response_column][i]:
            query = examples[prompt_column][i]
            if history_column is None or len(examples[history_column][i]) == 0:
                prompt = query
            else:
                prompt = ""
                history = examples[history_column][i]
                # 把对话历史拼成了 “[Round {}]\n问：{}\n答：{}\n” 格式
                for turn_idx, (old_query, response) in enumerate(history):
                    prompt += "[Round {}]\n问：{}\n答：{}\n".format(
                        turn_idx, old_query, response)
                # 当前轮拼成 “[Round {}]\n问：{}\n答：”
                prompt += "[Round {}]\n问：{}\n答：".format(len(history), query)
            inputs.append(prompt)
            targets.append(examples[response_column][i])

    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(
        inputs, max_length=data_args.max_source_length, truncation=True, padding=True)
    labels = tokenizer(text_target=targets,
                       max_length=max_target_length, truncation=True)

    if data_args.ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs
```

### 12.3、结果的评价方式


```python
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    score_dict = {
        "rouge-1": [],
        "rouge-2": [],
        "rouge-l": [],
        "bleu-4": []
    }
    for pred, label in zip(decoded_preds, decoded_labels):
        hypothesis = list(jieba.cut(pred))
        reference = list(jieba.cut(label))
        rouge = Rouge()
        scores = rouge.get_scores(' '.join(hypothesis), ' '.join(reference))
        result = scores[0]

        for k, v in result.items():
            score_dict[k].append(round(v["f"] * 100, 4))
        bleu_score = sentence_bleu([list(label)], list(
            pred), smoothing_function=SmoothingFunction().method3)
        score_dict["bleu-4"].append(round(bleu_score * 100, 4))

    for k, v in score_dict.items():
        score_dict[k] = float(np.mean(v))
    return score_dict
```

### 12.3.1、**知识点 1：**为什么不直接看 Loss

语言模型在训练过程中 Loss 是如何计算的

<img src="lm_loss.png" style="margin-left: 0px" width="800px">

考虑下面的情况：

<img src="lm_loss2.png" style="margin-left: 0px" width="400px">

相似度高，但是 loss 大，所以 loss 没法准确反应出我们真实的预期

### 12.3.2、**知识点 2：**生成模型常用的评价方法

- BLEU Score:
  - 计算输出与参照句之间的 n-gram 准确率（n=1...4）
  - 对短输出做惩罚
  - 在整个测试集上平均下述值

$\mathrm{BLEU}_4=\min\left(1,\frac{output-length}{reference-length}\right)\left(\prod_{i=1}^4 precision_i\right)^{\frac{1}{4}}$

- Rouge Score:

  - Rouge-N：将模型生成的结果和标准结果按 N-gram 拆分后，只计算召回率；
  - Rouge-L: 利用了最长公共子序列（Longest Common Sequence），计算：$P=\frac{LCS(c,r)}{len(c)}$, $R=\frac{LCS(c,r)}{len(r)}$, $F=\frac{(1+\beta^2)PR}{R+\beta^2P}$

- 对比 BLEU 与 ROUGE
  - BLEU 能评估流畅度，但指标偏向于较短的翻译结果（brevity penalty 没有想象中那么强）
  - ROUGE 不管流畅度，所以只适合深度学习的生成模型：结果都是流畅的前提下，ROUGE 反应参照句中多少内容被生成的句子包含（召回）

<div class="alert alert-warning">
<b>思考：</b> 为什么不能直接用BLEU或者ROUGE作为Loss函数？
</div>
 
### 12.3.3、编写训练代码

见附件

### 12.3.4、训练效果

**训练前**

| BLEU-4 | ROUGE-1 | ROUGE-2 | ROUGE-L | Whole Sent. Acc. |
| :----: | :-----: | :-----: | :-----: | :--------------: |
| 14.59  |  27.60  |  22.38  |  25.40  |      00.00       |

以上数字皆为百分数

**训练过程**

<img src="log.pt2.png" style="margin-left: 0px" width="800px">

**训练后**

| BLEU-4 | ROUGE-1 | ROUGE-2 | ROUGE-L | Whole Sent. Acc. |
| :----: | :-----: | :-----: | :-----: | :--------------: |
| 65.33  |  76.31  |  74.62  |  75.85  |      58.02       |

以上数字皆为百分数


## 十三、**实战 2：** 基于 LoRA 微调 ChatGLM-6B

### 13.1.1、编写训练代码

见附件

### 13.1.2、训练效果

**训练过程**

<img src="log.lora.png" style="margin-left: 0px" width="800px">

<div class="alert alert-warning">
<b>思考：</b> 是不是哪里不对？
</div>

### 13.1.3、学习率调节器实战


```python
class LoRASeq2SeqTrainer(Seq2SeqTrainer):
    def create_optimizer_and_scheduler(self, num_training_steps):
        """Set get_polynomial_decay_schedule_with_warmup power 2 for AdamW"""

        self.optimizer = AdamW(self.model.parameters(),
                               lr=self.args.learning_rate,
                               weight_decay=self.args.weight_decay)
        self.lr_scheduler = get_polynomial_decay_schedule_with_warmup(
            self.optimizer, 0, num_training_steps, power=2
        )
        
```

**训练过程**

<img src="log.lora.lr.png" style="margin-left: 0px" width="800px">

**训练后**

| BLEU-4 | ROUGE-1 | ROUGE-2 | ROUGE-L | Whole Sent. Acc. |
| :----: | :-----: | :-----: | :-----: | :--------------: |
| 64.84  |  75.79  |  74.04  |  75.32  |      57.77       |

以上数字皆为百分数


## 十四、**实战 3：** 基于上述方法微调 ChatGLM2-6B

**训练前**

| BLEU-4 | ROUGE-1 | ROUGE-2 | ROUGE-L | Whole Sent. Acc. |
| :----: | :-----: | :-----: | :-----: | :--------------: |
|  5.63  |  20.18  |  16.12  |  18.27  |       0.00       |

**训练代码**

见附件

**训练过程**

P-Tuning V2:

<img src="log.chatglm2.pt2.png" style="margin-left: 0px" width="800px">

LoRA:

<img src="log.chatglm2.lora.png" style="margin-left: 0px" width="800px">

**结果**

| Method      | BLEU-4 | ROUGE-1 | ROUGE-2 | ROUGE-L | Whole Sent. Acc. |
| :---------- | :----: | :-----: | :-----: | :-----: | :--------------: |
| P-Tuning V2 | 61.80  |  73.77  |  71.55  |  73.21  |      55.12       |
| LoRA        | 63.36  |  74.46  |  72.32  |  73.86  |      55.77       |


## 十五、即想要开放能力又想要垂直能力怎么办

**大模型的训练过程（宏观）**

<img src="train_llm.png" style="margin-left: 0px" width="800px">

**混合通用领域和垂直领域数据（举例：度小满--轩辕）**

<img src="xuanyuan.png" style="margin-left: 0px" width="800px">


## 作业：训练自己的模型

请跟随[实验指导](../lab/index.ipynb)完成本次作业。

<div class="alert alert-warning">
<b>注意：</b> 为了使大家能在合理时间内跑完实验，我们对数据集做了缩减
</div>

**注意观察训练中评估指标的变化**

<img src="train_log.png" style="margin-left: 0px" width="800px">

**训练后参考效果**

| Model       | Method      | BLEU-4 | ROUGE-1 | ROUGE-2 | ROUGE-L | Whole Sent. Acc. |
| :---------- | :---------- | :----: | :-----: | :-----: | :-----: | :--------------: |
| ChatGLM-6B  | P-Tuning V2 | 60.52  |  70.38  |  68.84  |  70.00  |       56.2       |
|             | LoRA        | 61.58  |  71.69  |  69.92  |  71.28  |       57.6       |
| ChatGLM2-6B | P-Tuning V2 | 56.69  |  67.10  |  64.90  |  66.43  |       51.2       |
|             | LoRA        | 60.25  |  70.22  |  68.56  |  69.58  |       56.4       |
